In [ ]:
import finlab
finlab.login(open("config.txt", "r").read())

from finlab import data
from finlab.data import indicator
import pandas as pd
import _beowFmt as fm
import time
today, s1 = time.strftime("%Y%m%d_%H%M", time.localtime()) , ""

iStart = -240
opens    = data.get("price:開盤價").astype(float)[iStart: ]  #移除 .fillna(0) 如果資料有空白自動填0，這會與XQ計算的不吻合。
closes   = data.get("price:收盤價").astype(float)[iStart: ]
ycloses   = closes.shift(1, axis=0)
# highs    = data.get("price:最高價").fillna(0).astype(float)[iStart: ]
# lows     = data.get("price:最低價").fillna(0).astype(float)[iStart: ]
volumns  = data.get("price:成交股數").fillna(0).astype(float)[iStart: ]/1000
volumns  = volumns.astype(int)
yv       = volumns.shift(1, axis = 0).fillna(0).astype(int)
cEma13 = closes.ewm(span=13, adjust=False).mean().round(3).astype(float)
cEma34 = closes.ewm(span=34, adjust=False).mean().round(3).astype(float)
cEma55 = closes.ewm(span=55, adjust=False).mean().round(3).astype(float)
cEma89 = closes.ewm(span=89, adjust=False).mean().round(3).astype(float)
cEma144 = closes.ewm(span=144, adjust=False).mean().round(3).astype(float)
cEma200 = closes.ewm(span=200, adjust=False).mean().round(3).astype(float)
# kd_k, kd_s  = data.indicator('STOCH')[iStart: ] 
# pd.set_option('display.expand_frame_repr', False)

# ## 三大法人資料 ###
# fcb     = data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')[-70: ]
# ic      = data.get('institutional_investors_trading_summary:投信買賣超股數')[-70: ]
# dealer  = data.get('institutional_investors_trading_summary:自營商買賣超股數(自行買賣)')[-70: ]
# dealer2 = data.get('institutional_investors_trading_summary:自營商買賣超股數(避險)')[-70: ]
# _v      = volumns.fillna(0).astype(float)[-70: ]

cond1 = closes > opens 
cond2 = opens > cEma34 
cond3_1 = cEma13 >= cEma34 
cond3_2 = cEma34 >= cEma55
cond3_3 = cEma55 >= cEma89
cond3_4 = cEma89 >= cEma144
cond3_5 = cEma144 >= cEma200
cond4   = volumns >= 2500


position = cond1 & cond3_1 & cond3_2 & cond3_3 & cond3_4 & cond3_5 & cond4
stockLists = fm.lastFinlabStockIdList(positionData = position) 
unwanted = ["2809","2880","2881","2882","2883","2884","2885","2886","2887","2888","2889","2890","2891","2892"] #清除不必要的金融股
filterdStock = [code for code in stockLists if code not in unwanted]
for c in filterdStock:
    s1 += f"{c}.TW,"
# fm.write_LogFile(f"xq_lab/{today}_跳過EMA3455均線.csv", s1)
fm.write_LogFile(f"xq_lab/{today}_均線多頭排列.csv", s1)


In [ ]:
stockId = '6654'
print(opens[stockId][-1:])
print(cEma55[stockId][-1:])
print(ycloses[stockId][-1:])
print(kd_k["2605"][-1:])

In [ ]:
from finlab.backtest import sim

closeW = closes.resample("W").last() #計算週K資料
closeM = closes.resample("M").last() #計算月K資料
c0 = closes.rise(2)
c1 = closeW.is_largest(15)
c2 = closeM.is_largest(15)

newhigh = closes.rolling(60, min_periods=1).max() == closes     # 近一日收盤價創近60日新高
d1 = (newhigh.shift(1) == 0).rolling(30).sum() > 0              # 前30日(不包含今日)收盤價價未創新高
d2 = (newhigh.shift(30).rolling(25).sum() > 0)                  # 第30日～第55日前的收盤價有至少一日收盤價創近60日新高


position = newhigh
position = position.loc[list(position.index)[::3]]
report = sim(position, resample='M', name="突破共振", upload=False)
report.display()

In [ ]:
from finlab.backtest import sim

ma5   = closes.ewm(span=5, adjust=False).mean().round(2)
ma7   = closes.ewm(span=7, adjust=False).mean().round(2)
ma10  = closes.ewm(span=10, adjust=False).mean().round(2)
ma20  = closes.ewm(span=20, adjust=False).mean().round(2)
ma60  = closes.ewm(span=60, adjust=False).mean().round(2)
ma240 = closes.ewm(span=240, adjust=False).mean().round(2)

amplitude = ((closes - opens) / opens) > 0.04
kbar = opens < closes
c0 = closes > ma5
c1 = ma5 > ma7
c2 = ma7 > ma10
c3 = ma10 > ma20
c4 = ma20 > ma60
c5 = ma60 > ma240


position = amplitude & kbar & c0 & c3 & c4 & c5
position = position.loc[list(position.index)[::3]]
report = sim(position, resample='M', name="TTTT444", upload=False)
report.display()


In [ ]:
#----取得股票名稱----
basic2 = data.get('company_basic_info')
dfff = basic2[["stock_id","公司簡稱","產業類別"]]
df_b = dfff.set_index('stock_id')
df_b

def getStockName(stockId):
    return df_b.loc[stockId]["公司簡稱"]

dff = position.iloc[-1:].transpose()
dff["stock"] = dff.index
dff.columns = ["val", "stock"]
df2 = dff[ (dff["val"] == True) & (dff["stock"].str.len() == 4)].dropna(axis = 0, how ='any')
df2.index.tolist()


import _beowFmt as fm 
today = position.iloc[-1:].index[0].strftime("%Y%m%d")
s1, s2 = '', ''

for c in df2.index.tolist():
    s1 += f"{c},"
    s2 += f"{c}-{getStockName(c)}\n"
fm.write_LogFile(f"datas-xq/{today}_均線之上_震幅超過4.csv", s1) 
fm.write_LogFile(f"datas/{today}_均線之上_震幅超過4.csv", s2)

In [ ]:
#二次創高
import pandas as pd
import numpy as np
from finlab import data
from finlab import backtest

close = data.get('price:收盤價')
vol = data.get('price:成交股數')
sma20 = close.average(20)
sma60 = close.average(60)  
newhigh = close.rolling(60, min_periods=1).max() == close   # 近一日收盤價創近60日新高
cond1 = newhigh
cond2 = (newhigh.shift(1) == 0).rolling(30).sum() > 0       # 前30日(不包含今日)收盤價價未創新高
cond3 = (newhigh.shift(30).rolling(25).sum() > 0)           # 第30日～第55日前的收盤價有至少一日收盤價創近60日新高
cond4 = (close.shift(30).rolling(25).max() < close)         # 前30日～55日的收盤價的最高 < 近日收盤價 (突破信號)
cond5 = close > close.shift(120)                            # 近日收盤價大於120前收盤價
cond6 = close > close.shift(60)                             # 近日收盤價大於60前收盤價
cond61= close > close.shift(30)                             # 近日收盤價大於60前收盤價!!!
rev = data.get('monthly_revenue:當月營收')
cond7 = rev.average(3) > rev.average(12)                    # 近3月平均營收大於12月平均營收
cond8 = vol.average(5) > vol.average(20)                    # 近5日平均成交量大於近20日平均成交量
cond9 = sma60 > sma60.shift()                               # 季線下彎了?


rsi20 = data.indicator("RSI", timeperiod=20)
rsi60 = data.indicator("RSI", timeperiod=60)
rsi120 = data.indicator("RSI", timeperiod=120)
ROE稅後 = data.get("fundamental_features:ROE稅後")

長週期上漲 = rsi120 > 55
中週期別過熱 = rsi60 < 75
短週期RSI上漲 = rsi20.pct_change(3) > 0.02
短週期RSI高檔頓化 = (rsi20 > 75).sustain(3)
ROE為正 = ROE稅後 > 0

buy = (cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond61 & cond7 & cond8 & cond9 
     & 長週期上漲 
     & 中週期別過熱 
     & 短週期RSI上漲 
     & 短週期RSI高檔頓化
     & ROE為正)
sell = sma20 < sma20.shift()                                # 月線下彎就賣出                  

position = pd.DataFrame(np.nan, index=buy.index, columns=buy.columns)
position[buy] = 1
position[sell] = 0
position = position.ffill().fillna(0)

report = backtest.sim(position.loc['2014':], resample="D",name='二次創高股票_', live_performance_start='2018-06-01')

In [ ]:
pd.set_option('display.max_rows', 500)

df4 = report.get_trades()
df5 = df4[df4["exit_date"].isna() & (~df4["entry_date"].isna()) & (df4["pdays"] > 0) & (df4["pdays"] < 100)].sort_values(by='pdays', ascending=False)
df5[["stock_ids","stock_name"]] = df5["stock_id"].str.split(" ", 1, expand = True)

import _beowFmt as fm
s1, s2, today = '', '', position.iloc[-1:].index[0].strftime("%Y%m%d")

for c in df5["stock_ids"].tolist():
    s1 += f"{c}.TW,"
    s2 += f"{c}-{getStockName(c)}\n"
fm.write_LogFile(f"datas-xq/{today}_二次創新高股票_Rsi3.csv", s1)
fm.write_LogFile(f"datas/{today}_二次創新高股票_Rsi3.csv", s2)

In [ ]:
import pandas as pd
import numpy as np
import math
import _beowSet as bs  
import _beowFmt as fm 

np.warnings.filterwarnings("ignore")

def genInsertStockParasSQL(stockId):
    iStart = -250

    open    = opens[stockId].fillna(0).astype(float)[iStart: ]      #如果資料有空白自動填0
    c       = closes[stockId].fillna(0).astype(float)[iStart: ]
    high    = highs[stockId].fillna(0).astype(float)[iStart: ]
    low     = lows[stockId].fillna(0).astype(float)[iStart: ]
    volumn  = volumns[stockId].fillna(0).dropna().astype(float)[iStart: ]

    if (c[-1:].values[0] == 0.0 and high[-1:].values[0] == 0.0 and volumn[-1:].values[0] == 0.0 and low[-1:].values[0] == 0.0 ):
        return []  

    ma5   = c.ewm(span=5, adjust=False).mean().round(2)                         #以Series來做四捨五入
    ma7   = c.ewm(span=7, adjust=False).mean().round(2)
    ma10  = c.ewm(span=10, adjust=False).mean().round(2)
    ma20  = c.ewm(span=20, adjust=False).mean().round(2)
    ma60  = c.ewm(span=60, adjust=False).mean().round(2)
    ma240 = c.ewm(span=240, adjust=False).mean().round(2)
    kbar  = (c > open)
    kbar  = kbar.apply(lambda x: "R" if x else "G")
    
    yc = c.shift(1, axis = 0)               #昨天的收盤價, shift往後移一天，超級方便!!
    yh = high.shift(1, axis = 0)            #昨天的最高價, shift往後移一天，超級方便!!
    yl = low.shift(1, axis = 0)             #昨天的最低價, shift往後移一天，超級方便!!

    dff = pd.DataFrame({
        "kbar": kbar, "open" :open, "high" : high, "low" : low, "close" : c, 
        "yc": yc, "yh":yh, "yl": yl, "volumn": volumn,
        "ma5" : ma5, "ma7" : ma7, "ma10": ma10, "ma20": ma20, "ma60": ma60, "ma240": ma240
    })
    
    dff["volumn"]       = dff["volumn"].apply(lambda x : 1 if( math.isnan(x) == True ) else int(x/1000))    #張數  
    dff["kbar_top"]     = dff.apply(fm.setKbarTop,    axis = 1)
    dff["kbar_top_y"]   = dff["kbar_top"].shift(1, axis = 0)                                                #昨天的K棒最高點 
    dff["kbar_bottom"]  = dff.apply(fm.setKbarBottom, axis = 1)
    dff["upper-shadow"] = dff.apply(fm.fmtUpperShadow,axis = 1) 
    dff["lower-shadow"] = dff.apply(fm.fmtLowerShadow,axis = 1) 
    dff["kbar-body"]    = dff.apply(fm.fmtKbarBody,   axis = 1)
    dff["amplitude"]    = dff.apply(fm.fmtAmplitude,  axis = 1)  
    dff["iCtnC"]        = dff.apply(fm.fmtCtnC,       axis = 1)
    dff["iGap"]         = dff.apply(fm.fmtIGap,       axis = 1)
    dff["ma_dscr"]      = dff.apply(fm.diffRatio,     axis = 1)
    dff["d_ma5"]        = dff.apply(fm.isCrossMA5,    axis = 1)
    dff["d_ma10"]       = dff.apply(fm.isCrossMA10,   axis = 1)
    dff["d_ma20"]       = dff.apply(fm.isCrossMA20,   axis = 1)
    dff["d_ma60"]       = dff.apply(fm.isCrossMA60,   axis = 1)
    dff["d_ma240"]      = dff.apply(fm.isCrossMA240,  axis = 1)

    dff["sMa5"]         = dff["ma5"] - dff["ma5"].shift(1, axis =0)
    dff["sMa5a"]        = round(dff["sMa5"] / dff["ma5"].shift(1, axis =0) * 100, 2)
    dff["sMa5"]         = dff["sMa5"].apply(lambda x: 1 if (x > 0 ) else -1)
    dff["sMa5y"]        = dff["sMa5"].shift(1, axis =0)
    dff["crvFlagMa5"]   = dff["sMa5"] * dff["sMa5"].shift(1, axis = 0)
    dff["maF5"]         = dff.apply(fm.fmtGetCurvHookMa5, axis = 1)

    dff["sMa10"]         = dff["ma10"] - dff["ma10"].shift(1, axis =0)
    dff["sMa10a"]        = round(dff["sMa10"] / dff["ma10"].shift(1, axis =0) * 100, 2)
    dff["sMa10"]         = dff["sMa10"].apply(lambda x: 1 if (x > 0 ) else -1)
    dff["sMa10y"]        = dff["sMa10"].shift(1, axis =0)
    dff["crvFlagMa10"]   = dff["sMa10"] * dff["sMa10"].shift(1, axis = 0)
    dff["maF10"]         = dff.apply(fm.fmtGetCurvHookMa10, axis = 1)

    dff["sMa20"]         = dff["ma20"] - dff["ma20"].shift(1, axis =0)
    dff["sMa20a"]        = round(dff["sMa20"] / dff["ma20"].shift(1, axis =0) * 100, 2)
    dff["sMa20"]         = dff["sMa20a"].apply(lambda x: 1 if (x > 0 ) else -1)
    dff["sMa20y"]        = dff["sMa20"].shift(1, axis =0)
    dff["crvFlagMa20"]   = dff["sMa20"] * dff["sMa20"].shift(1, axis = 0)
    dff["maF20"]         = dff.apply(fm.fmtGetCurvHookMa20, axis = 1)

    dff["sMa60"]         = dff["ma60"] - dff["ma60"].shift(1, axis =0)
    dff["sMa60a"]        = round(dff["sMa60"] / dff["ma60"].shift(1, axis =0) * 100, 2)
    dff["sMa60"]         = dff["sMa60"].apply(lambda x: 1 if (x > 0 ) else -1)
    dff["sMa60y"]        = dff["sMa60"].shift(1, axis =0)
    dff["crvFlagMa60"]   = dff["sMa60"] * dff["sMa60"].shift(1, axis = 0)
    dff["maF60"]         = dff.apply(fm.fmtGetCurvHookMa60, axis = 1)

    dff["sMa240"]        = dff["ma240"] - dff["ma240"].shift(1, axis =0)
    dff["sMa240a"]        = round(dff["sMa240"] / dff["ma240"].shift(1, axis =0) * 100, 2)
    dff["sMa240"]        = dff["sMa240"].apply(lambda x: 1 if (x > 0 ) else -1)
    dff["sMa240y"]       = dff["sMa240"].shift(1, axis =0)
    dff["crvFlagMa240"]  = dff["sMa240"] * dff["sMa240"].shift(1, axis = 0)
    dff["maF240"]        = dff.apply(fm.fmtGetCurvHookMa240, axis = 1)

    # 改善dtypes來提升效能
    dff = dff.astype({"kbar":"category","ma_dscr":"category","d_ma5":"category","d_ma5":"category","d_ma10":"category","d_ma20":"category","d_ma60":"category","d_ma240":"category","maF5":"category","maF10":"category","maF20":"category","maF60":"category","maF240":"category"})
    # print(dff.dtypes)
    # print(dff.tail(10))
    k, d = round(kd_k[stockId], 2), round(kd_s[stockId], 2)


    df = dff[-2:].copy()                            # 資料若不copy會引發 Returning a view versus a copy 的錯誤。
    df[["stockId","k","d"]] = [stockId, k, d]
    df["today"]             = df.index.date[0].strftime("%Y-%m-%d")
    df["sql_Paras"]         = df.apply(fm.fmtSql_stockParas, axis = 1)
    df["sql_MA"]            = df.apply(fm.fmtSql_stockMA   , axis = 1)
    sql_Paras               = df.iloc[0]["sql_Paras"]
    sql_MA                  = df.iloc[0]["sql_MA"]
    return [] if(df["ma5"].all()==0) else [sql_Paras, sql_MA]

a = genInsertStockParasSQL("2330")
print(a)

In [ ]:
import pandas as pd
import numpy as np
import math
import _beowSet as bs  
import _beowFmt as fm 
np.warnings.filterwarnings("ignore")

sqlkeep, i, sql_Paras, sql_MA, arr, brr, stockIds = "", 0, "", "", [], [], opens.columns.values.tolist()
new_stockIds = list(filter(lambda x : len(x) == 4, stockIds))
nowDate = closes["1101"].fillna(0).astype(float).tail(1).index.date[0]
today = nowDate.strftime("%Y-%m-%d")

try:
    for stockId in new_stockIds:
        i += 1
        ss = genInsertStockParasSQL(stockId)
        
        if (len(ss) > 0):
            arr.append(ss[0])
            brr.append(ss[1])
            
            if (i % 50 == 0):    
                sql_Paras = "\n insert into dbo.StockParas (stockId, closeDate,kbar, v, o, c, h, l, yc, head, body, footer, iGap,amplitude,k,d) values \n" + "\n,".join(arr)
                sql_MA    = "\n insert into dbo.StockMA (stockId, closeDate,kbar,ma_dscr, ma5, ma7, ma10, ma20, ma60, ma240, d_ma5, d_ma10, d_ma20, d_ma60, d_ma240, maF5, maF10, maF20, maF60, maF240, maS5, maS10, maS20, maS60, maS240) values \n" + "\n,".join(brr)
                sql = sql_Paras +"\n"+ sql_MA
                print("目前匯入筆數" + str(i) , end='\r')
                # sqlkeep += sql
                # fm.write_LogFile(f"sqlback/{today}_debug.sql", sql)
                bs.InsertIntoMSSQL2017(sql) #一次輸入給SQL新增。更快!
                sql_Paras, sql_MA, arr, brr = "", "", [], []

    sql_Paras = "\n insert into dbo.StockParas (stockId, closeDate,kbar, v, o, c, h, l, yc, head, body, footer, iGap,amplitude,k,d) values \n" + "\n,".join(arr)
    sql_MA    = "\n insert into dbo.StockMA (stockId, closeDate,kbar,ma_dscr, ma5, ma7, ma10, ma20, ma60, ma240, d_ma5, d_ma10, d_ma20, d_ma60, d_ma240, maF5, maF10, maF20, maF60, maF240, maS5, maS10, maS20, maS60, maS240) values \n" + "\n,".join(brr)
    sql = sql_Paras +"\n"+ sql_MA
    print("目前匯入筆數" + str(i) , end='\r')
    # sqlkeep += sql
    bs.InsertIntoMSSQL2017(sql) #一次輸入給SQL新增。更快!
    sql_Paras, sql_MA, arr, brr = "", "", [], []
except Exception as e:
    print(e)
    fm.write_LogFile(f"sqlback/{today}_debug.sql", sql)
    # fm.write_LogFile(f"sqlback/{today}_paras.sql", sqlkeep)
    # sqlkeep = ""


In [ ]:
pd.set_option('display.expand_frame_repr', False)

## 三大法人資料 ###
# fcb     = data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')
# ic      = data.get('institutional_investors_trading_summary:投信買賣超股數')
# dealer  = data.get('institutional_investors_trading_summary:自營商買賣超股數(自行買賣)')
# dealer2 = data.get('institutional_investors_trading_summary:自營商買賣超股數(避險)')
# _v       = data.get('price:成交股數').fillna(0).astype(float)[-70:]

nowDate = _v["1101"].fillna(0).astype(float).tail(1).index.date[0]
today = nowDate.strftime("%Y-%m-%d")
print(today)
v      = (_v/1000).round().astype(int).fillna(0) 
yv    = v.shift(1, axis = 0).fillna(0).astype(int)
vma5  = v.ewm(span=5, adjust=False).mean().round().astype(int).fillna(0)
vma20 = v.ewm(span=20, adjust=False).mean().round().astype(int).fillna(0)
vma60 = v.ewm(span=60, adjust=False).mean().round().astype(int).fillna(0)

# print(fcb[fcb.index.get_level_values("date") == today])

_fcb    = fcb[fcb.index.get_level_values("date") == today].transpose()
_ic     = ic[ic.index.get_level_values("date") == today].transpose()
_dealer = dealer[dealer.index.get_level_values("date") == today].transpose()
_dealer2= dealer2[dealer2.index.get_level_values("date") == today].transpose()
_dealer = _dealer + _dealer2

dfp =pd.DataFrame({
    "_fc"  : _fcb[today]/1000,
    "_ic"  : _ic[today]/1000,
    "_dc"  : _dealer[today]/1000,
    "yv"   : yv.loc[today],
    "v"    : v.loc[today], 
    "ma5"  : vma5.loc[today], 
    "ma20" : vma20.loc[today], 
    "ma60" : vma60.loc[today]
})
dfp = dfp.dropna(axis = 0, how ='any')
dfp[["stock_id","dt"]] = [dfp.index, today]
dfp["sql_volumn"] = dfp.apply(fm.fmtSql_Volumn, axis= 1)    #產出SQL指令碼

svaccum, sql_volumn = "",""                                 #切成100筆一份同時 insert到MSSQL當中。
for i in range(1,17):
    a, b = (i-1)*100, i*100
    sql_volumn = "insert into volumn (stockId,closeDate,fc,ic,dealer,accu_fc,accu_ic,k_fc,k_ic,yv,v,ma5,ma20,ma60) values " + ",".join(dfp[dfp["stock_id"].str.len() == 4 ]["sql_volumn"][a:b].tolist())
    svaccum += sql_volumn
    bs.InsertIntoMSSQL2017(sql_volumn)                      #一次輸入給SQL新增。更快!

fm.write_LogFile(f"sqlback/{today}_volumns.sql", svaccum)

bs.proc_final_SqlScript2(today)    


In [ ]:
# data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')[-5:]

with data.universe(market='TSE_OTC'):

    研究發展費 = data.get("financial_statement:研究發展費").deadline()
    營業收入淨額 = data.get("financial_statement:營業收入淨額").deadline()
    research_ratio= 研究發展費 / 營業收入淨額 * 100
    research_ratio_drop = research_ratio[研究發展費 < 營業收入淨額]
    research_ratio_drop

research_ratio_drop

In [ ]:
## 找出盤整很久的 ##
import pandas as pd
import numpy as np
from finlab import data
from finlab import backtest

close = data.get('price:收盤價')
vol = data.get('price:成交股數')
sma20 = close.average(20)
sma60 = close.average(60)
sma240= close.average(240)  
sma480 = close.average(480)

cond1m = sma20 > sma20.shift()
cond1q = sma60 > sma60.shift()
cond1y = sma240> sma240.shift()
cond2m = sma20 > sma60
cond2q = sma60 > sma240
cond2yy = sma240 > sma480
cond3y = vol.average(240) < vol
cond3q = vol.average(60) < vol
cond4 = close.rise().sustain(2)

position = cond1m & cond1q & cond1y & cond2m & cond2q & cond2yy & cond3y & cond3q & cond4

dff = position[-1:].transpose()
dff["stock"] = dff.index
dff.columns = ["val", "stock_id"]
df3 = dff[ (dff["val"] == True) & (dff["stock_id"].str.len() == 4)].dropna()

print(df3, len(df3))

import _beowFmt as fm 
s1, s2, today = '', '', position.iloc[-1:].index[0].strftime("%Y%m%d") 

for c in df3["stock_id"].tolist():
    s1 += f"{c}.TW,"
    s2 += f"{c}-{getStockName(c)}\n"
fm.write_LogFile(f"datas-xq/{today}_大均線多頭且往上鉤.csv", s1)
fm.write_LogFile(f"datas/{today}_大均線多頭且往上鉤.csv", s2)

In [32]:
import finlab
finlab.login(open("config.txt", "r").read())

from finlab import data

# 使用 FinLab 獲取台灣股市所有股票的基本資訊
# stock_list = data.stock_id

# 獲取股票的族群資訊（以台灣股市為例，假設族群分類為"產業別"）
df = data.get('security_industry_themes')

# print(industry_info["category"])

ss = ''
rootpath= "D:/project/finlabexportdata"
js_file_path = f"{rootpath}/data/stock_category.json"
for i, row in industry_info.iterrows():
    stock_id = row["stock_id"]
    category = row["category"][:30]
    # 將每一行資料按照特定格式串接到 ss 字串上
    ss += f'{{"stockId": {stock_id}, "cg": "{category}"}},'

# 打开 JavaScript 文件以写入数据
with open(js_file_path, 'w', encoding='utf-8') as js_file:
    js_file.write(f"[{ss}]")


# 将 DataFrame 转换为 JSON 数据并追加到 JavaScript 文件中
# df_combined3.to_json(js_file_path, orient='records', lines=False, default_handler=None)




輸入成功!


In [1]:

# 找出提早公布月營收資料的股票
import finlab
finlab.login(open("config.txt", "r").read())

from finlab import data
df = data.get('monthly_revenue:當月營收')
import _beowFmt as fm 
import pandas as pd
pd.set_option('display.expand_frame_repr', False)


# 找出最後一個月份的日期
dfm = df.tail(2).to_period('M')
print(dfm)
dfm2 = dfm.shift(1)

not_nan_columns = dfm.columns[dfm.notna().all()]

print(not_nan_columns)
ss=''
for c in not_nan_columns:
    ss += f'{c},'
fm.write_LogFile(f"data/json/monthlyRevenue.json", f"[{ss[:-1]}]") 



輸入成功!
symbol   000116  000930  000960  0009A0       1101       1102      1103      1104  1107      1108  ...      9944       9945      9946     9949      9950      9951     9955       9958     9960      9962
date                                                                                               ...                                                                                                   
2023-12     NaN     NaN     NaN     NaN  8786909.0  6915817.0  255428.0  777614.0   NaN  503715.0  ...  155920.0  2653061.0  105001.0  17917.0  173879.0  519328.0  86398.0  1546059.0  92299.0  275411.0
2024-01     NaN     NaN     NaN     NaN        NaN        NaN       NaN       NaN   NaN       NaN  ...       NaN        NaN       NaN      NaN       NaN       NaN      NaN        NaN      NaN  432453.0

[2 rows x 2124 columns]
Index(['1213', '1220', '1225', '1236', '1256', '1264', '1304', '1305', '1308',
       '1309',
       ...
       '8940', '8941', '8942', '8996', '9802', '9906', '

In [7]:
# %%
#前置作業準備，只要一次!!
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import _beowFmt as fm 

from finlab import data
import os

rootpath= "D:/project/finlabexportdata/"

yesterday = fm.getLastFileDate(f"{rootpath}/volumeData", "ma_")

df5k = pd.read_json(f"{rootpath}paras/mapping5K.json")
dfv = pd.read_csv(f"{rootpath}volumeData/ma_{yesterday}.csv")
dfv["id"] = dfv["id"].astype("string")
# dfv = dfv.drop(columns=['close'])
#print(dfv[dfv["id"] == "3265"])

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

url4 = "https://www.wantgoo.com/investrue/all-alive"
r3 = requests.get(url4, headers = headers).content
soup = BeautifulSoup(r3, "html.parser")
rr3 = soup.prettify()
dfn = pd.read_json(rr3)
dfn = dfn[(dfn["id"]>="1101") & (dfn["id"]<="9999") & (dfn["id"].str.len() == 4)]
dfn = dfn.drop(columns=['type','country','url', 'industries'])
dfn["id"] = dfn["id"].astype("string")

def comp_stockName(r):
    id, name = r["id"], r["name"]
    return f"{id},{name}"

dfn["comp"] = dfn.apply(comp_stockName, axis = 1)
stockNameStr = ""
for c in dfn["comp"].tolist():
    stockNameStr += f"{c}\n"
fm.write_LogFile(f"{rootpath}paras/股票名稱.csv", stockNameStr)  

### 計算量比參數 ###
import time
def getRate5K():
    localtime = time.localtime()
    nowTime = time.strftime("%H:%M", localtime)
    Rate5k = df5k[(df5k["e"] >= nowTime) & (df5k["b"] <= nowTime)]["w"].values[0]  #五分K量比預估放大參數 
    print(f"目前時間：{nowTime},預估量放大係數:{Rate5k}")
    return Rate5k

# print(getRate5K())

from IPython.display import display

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.width', 180)                       # 设置打印宽度(**重要**)
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

r = requests.get("https://www.wantgoo.com/investrue/all-quote-info", headers = headers).content
soup = BeautifulSoup(r, "html.parser")
rr = soup.prettify()
df = pd.read_json(rr)
df = df.drop(columns=['tradeDate', 'time','millionAmount'])
df["id"] = df["id"].astype("string")
df = pd.merge(df, dfn, left_on="id", right_on="id")  ## 結合股票名稱
df["float"] = df["close"].astype("float")
df["amp"] = ((df["close"] - df["previousClose"])/df["previousClose"] * 100).round(2)
df["jump"] = df["open"] - df["previousClose"]
df["jumpRate"] = (df["jump"] / df["open"] * 100).round(2)
df = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["id"].str.len() == 4) & (df["close"] > df["previousClose"])].sort_values("jumpRate", ascending=False)
# df.info()

print(df)

最新的檔案日期： 20240108
        id     flat    floor     ceil     open     high      low    close  volume  previousClose  previousVolume  previousMillionAmount    name    market         comp    float    amp   jump  jumpRate
583   3499    20.60    18.55    22.65    22.65    22.65    22.65    22.65     939          20.60           282.0                   5.85     環天科       OTC     3499,環天科    22.65   9.95   2.05      9.05
759   5310    20.60    18.55    22.65    22.65    22.65    22.00    22.65     118          20.60            14.0                   0.29      天剛       OTC      5310,天剛    22.65   9.95   2.05      9.05
625   3622    42.25    38.05    46.45    46.00    46.20    44.35    45.10    6963          42.25          1016.0                  43.54      洋華    Listed      3622,洋華    45.10   6.75   3.75      8.15
102   2427    24.80    22.35    27.25    27.00    27.00    25.40    25.60    5434          24.80           871.0                  21.55     三商電    Listed     2427,三商電    25.60   3.23